### Ноутбук строит карту используя геоданные, визуализируя неэпидемологические данные

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
from urllib.request import urlopen
import json
import plotly.graph_objects as go

In [3]:
pd.set_option("display.max_columns", 35)

In [4]:
epi_df = pd.read_csv('./data/export_EPI_MAIN_weekly_201540_201939_FINAL__map_features.csv', index_col=0)

In [5]:
threshold = pd.read_csv('./data/threshold.csv', header=None)
threshold.columns = ['DISTRICT_NAME', 'threshold']

In [6]:
epi_df_with_threshold = pd.merge(left=epi_df, right=threshold, how='left')

In [7]:
from datetime import date, datetime

def create_dateime_col(full_df, day):
    return pd.to_datetime(full_df[['YEAR', 'WEEK']].apply(
        lambda x: date.fromisocalendar(x[0], x[1], day), axis=1))

### mapbox

In [8]:
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/russia.geojson') as response:
    counties = json.load(response)
    
for k in range(len(counties['features'])):
    counties['features'][k]['id'] = k

In [9]:
region_id_list = []
regions_list = []

for k in range(len(counties['features'])):
    region_id_list.append(counties['features'][k]['id'])
    regions_list.append(counties['features'][k]['properties']['name'])

df_regions = pd.DataFrame()
df_regions['region_id'] = region_id_list
df_regions['region_name'] = regions_list

In [10]:
map1 = go.Choroplethmapbox(geojson=counties,
                           locations=df_regions['region_id'],
#                            z=df_regions['region_id'],
                           z=[4]*df_regions['region_id'].size,
                           text=df_regions['region_name'],
                           colorscale=[[0, 'rgb(34, 150, 79)'],
                                       [0.2, 'rgb(249, 247, 174)'],
                                       [0.8, 'rgb(253, 172, 99)'],
                                       [1, 'rgb(212, 50, 44)']],
                           colorbar_thickness=20,
#                            customdata=np.stack([df['cases'], df['died'], df['sick'], df['healed']], axis=-1),
#                            hovertemplate='<b>%{text}</b>'+ '<br>' +
#                                          'New cases: %{z}' + '<br>' +
#                                          'Active cases: %{customdata[0]}' + '<br>' +
#                                          'Deaths: %{customdata[1]}' + '<br>' +
#                                          'Total cases: %{customdata[2]}' + '<br>' +
#                                          'Recovered: %{customdata[3]}' +
#                                          '<extra></extra>',
                           hoverinfo='text, z',
                           marker_line_color="black",
                           marker_line_width=0.1
)

In [11]:
map2 = go.Scattermapbox(
        lat=epi_df_with_threshold['Широта'],
        lon=epi_df_with_threshold['Долгота'],
        text =epi_df_with_threshold["DISTRICT_NAME"],
        mode = 'markers',
        marker_color='rgba(43, 234, 23, 0.5)',
#         marker_color = df['cnt'],
        marker_size = 1 + epi_df_with_threshold['threshold'])
        )

IndentationError: unexpected indent (<ipython-input-11-8d7ac0ce2473>, line 9)

In [ ]:
epi_df_with_threshold

In [ ]:
# epi_df['start_date'] = create_dateime_col(epi_df, 1)
# epi_df['end_date'] = create_dateime_col(epi_df, 7)

In [ ]:
fig = go.Figure(data=[map1, map2])

In [ ]:
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=1, mapbox_center = {"lat": 66, "lon": 94})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()